### Hybrid Search Langchain and Pinecone

In [1]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
from langchain_community.retrievers import PineconeHybridSearchRetriever

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")

index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index=pc.Index(index_name)

/home/vpsr/Desktop/python/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [3]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()

[nltk_data] Downloading package stopwords to /home/vpsr/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 37.64it/s]


In [5]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [6]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


In [11]:
retriever.invoke("What city did i visit in 2022")

[Document(metadata={'score': 0.509391725}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.357693106}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.336136639}, page_content='In 2021, I visited New Orleans')]